In [18]:
# over welke periode gaat het dataframe: 10 jaar
# aggregratie op jaar niveau, eventueel later op week niveau

# welke X-en:
# PC4
# LON 
# LAT
# Res Regio
# Gemeente
# tem: hoe warm was het jaar?

# sjv_totaal
# factor_maand
# factor
# week_verbruik_kw
# laadpalen
# electrische auto's
# warmtepompen
# Opwekking zonneenergie
# Opwekking Windenergie
# Decentrale opwekgegevens
# Knelpunt indicatie (Provincie?)

In [19]:
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import seaborn as sns

In [20]:
# show all columns in the dataframe
pd.set_option('max_columns', None)

In [21]:
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

# Alle benodige ht5 dataframes inlezen

In [22]:
# decentrale_opwekkingsgegevens gegevens inlezen
df_opwek = pd.read_hdf('decentrale_opwekkingsgegevens_data.h5')
# kleinverbruikgegevens gegevens inlezen
df_verbruik = pd.read_hdf('kleinverbruikgegevens_data.h5')
# mapping van CBS buurt naar PC4
df_cbs_buurt_pc4 = pd.read_hdf('cbs_buurt_pc4.h5')
# mapping van PC4 buurt naar RES regio
df_pc4_res = pd.read_hdf('pc4_res.h5')
# cbs data
df_cbs = pd.read_hdf('cbs_data.h5')

# Nieuw master dataframe maken

In [23]:
# maak nieuw dataframe met de jaren
df = pd.DataFrame(pd.date_range(start='1/1/2010', end='31/12/2020' , freq='Y'), columns=['DATUM']).sort_values(by='DATUM', ascending=False)
df['JAAR'] = df['DATUM'].dt.year

# Vebruik toevoegen aan dataframe

In [24]:
# df_verbruik is nog per pc6, dus eerst omzetten naar pc4
df_verbruik_pc4 = df_verbruik.groupby(['PC4','JAAR']
                                      , as_index=False
                                     ).agg({'SJV_TOTAAL': 'sum'
                                            ,'AANSLUITINGEN_AANTAL': 'sum'  
                                            ,'LEVERINGSRICHTING_PERC': 'mean'
                                           }
                                          )

# join df met df_verbruik
df = pd.merge(df, df_verbruik_pc4, on='JAAR', how='left')

# Opwek toevoegen aan dataframe

In [25]:
# jaar toevoegen
# 2020-07-01 verwijderen, de maand van de peildatum moet januari zijn
df_opwek = df_opwek[df_opwek['Peildatum'].dt.month == 1]
# 2020-01-01 wordt 2019, 2021-01-01 wordt 2020
df_opwek['JAAR'] = df_opwek['Peildatum'].dt.year - 1
# PC4 toevoegen
df_cbs_buurt_pc4 = df_cbs_buurt_pc4.rename(columns={'Buurt2020': 'CBS Buurtcode'})
df_opwek = pd.merge(df_opwek, df_cbs_buurt_pc4, on='CBS Buurtcode', how='left', suffixes=('','_2'))
# df_opwek group by PC4 en JAAR
df_opwek_pc4 = df_opwek.groupby(['PC4','JAAR'], as_index=False).agg({'Gemeente':'max'
                                                                     , 'Aantal aansluitingen in CBS-buurt': 'sum'
                                                                     , 'Aantal aansluitingen met opwekinstallatie':'sum'
                                                                     , 'Opgesteld vermogen':'sum'
                                                                    }
                                                                   )
# rename columns
df_opwek_pc4 = df_opwek_pc4.rename(columns={'Aantal aansluitingen in CBS-buurt': 'AANTAL AANSLUITINGEN IN PC4'
                                            , 'Gemeente': 'GEMEENTE'
                                            , 'Aantal aansluitingen met opwekinstallatie': 'AANTAL AANSLUITINGEN MET OPWEKINSTALLATIE'
                                            , 'Opgesteld vermogen': 'OPGESTELD VERMOGEN'
                                           }
                                  )

# join df met df_opwek_pc4
df = pd.merge(df, df_opwek_pc4, on=['JAAR','PC4'], how='left')

# RES regio toevoegen aan dataframe

In [26]:
df = pd.merge(df, df_pc4_res, on=['PC4'], how='left')

In [27]:
#df.info()

# CBS data voor 2015-2019 toevoegen. Selectie van relevante features nog niet  gedaan¶

In [28]:
df_cbs['PC4'].head()

0    1011
1    1012
2    1013
3    1014
4    1015
Name: PC4, dtype: int64

In [29]:
df_cbs.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
PC4,20308.0,5669.206421,2671.038412,1011.0,3328.0,5702.0,8043.0,9999.0
INWONER,20308.0,3619.416831,9000.093396,-99997.0,660.0,2650.0,6910.0,28410.0
MAN,20308.0,1421.189679,8552.518942,-99997.0,340.0,1320.0,3395.0,14195.0
VROUW,20308.0,1371.024769,9057.870561,-99997.0,320.0,1315.0,3490.0,14215.0
INW_014,20308.0,-1821.440467,15758.688512,-99997.0,105.0,410.0,1090.0,6535.0
...,...,...,...,...,...,...,...,...
STED,20308.0,-168.565097,4148.011899,-99997.0,2.0,4.0,5.0,5.0
JAAR,20308.0,2017.002019,1.414055,2015.0,2016.0,2017.0,2018.0,2019.0
WON_1524,16255.0,-56090.863796,49659.502833,-99997.0,-99997.0,-99997.0,25.0,2980.0
WOZWONING,12187.0,-2885.876344,17422.649913,-99997.0,178.0,225.0,275.0,1231.0


In [30]:
# Merge cbs data for years 2015-2019. Other years to be loacted or constructed 
df = pd.merge(df, df_cbs, on=['JAAR','PC4'], how='left')

In [31]:
# Review results
pd.options.display.max_rows = 500
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
JAAR,17130.0,2015.010274,3.168513,2010.000,2012.000000,2015.00000,2018.000000,2020.0
PC4,17130.0,7175.969527,1770.313330,4251.000,5563.000000,7429.00000,8917.000000,9999.0
SJV_TOTAAL,17130.0,7419.743900,7319.597724,26.000,1617.250000,4894.50000,11216.750000,52390.0
AANSLUITINGEN_AANTAL,17130.0,1609.624635,1679.368256,10.000,259.000000,902.50000,2607.000000,10063.0
LEVERINGSRICHTING_PERC,17130.0,93.662165,7.712743,26.895,90.444444,96.42182,99.693618,100.0
AANTAL AANSLUITINGEN IN PC4,2435.0,1867.102259,1774.069860,17.000,418.000000,1213.00000,2914.500000,8897.0
AANTAL AANSLUITINGEN MET OPWEKINSTALLATIE,2435.0,304.359754,301.218239,10.000,81.000000,205.00000,438.000000,2475.0
OPGESTELD VERMOGEN,2435.0,1330.798768,1217.099305,33.000,447.000000,955.00000,1848.000000,8278.0
INWONER,7731.0,3599.881387,4707.024553,-99997.000,600.000000,2185.00000,6017.500000,23005.0
MAN,7731.0,1654.812702,4706.014075,-99997.000,315.000000,1100.00000,2965.000000,11495.0


In [32]:
df[df['JAAR']==2019].head().transpose()

,1578,1579,1580,1581,1582
DATUM,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00
JAAR,2019,2019,2019,2019,2019
PC4,4251,4254,4255,4261,4264
SJV_TOTAAL,22391.0,10609.0,6383.0,12843.0,7366.0
AANSLUITINGEN_AANTAL,4720,2446,1630,2495,1284
LEVERINGSRICHTING_PERC,93.148667,86.7605,84.559487,92.70069,95.191429
GEMEENTE,Altena,NaN,NaN,Altena,Altena
AANTAL AANSLUITINGEN IN PC4,1922.0,NaN,NaN,289.0,1179.0
AANTAL AANSLUITINGEN MET OPWEKINSTALLATIE,117.0,NaN,NaN,39.0,76.0
OPGESTELD VERMOGEN,335.0,NaN,NaN,197.0,383.0


# Save dataframe 

In [33]:
# data locatie om de ht5 file op te slaan
data_processed_location = '../processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [34]:
try:
    os.remove('master_data.h5')
except:
    print ('File nog in gebruik of niet gevonden')

store = pd.HDFStore('master_data.h5')
store['master_data'] = df
store.close()

C:\Users\MarcelKoolwijk\anaconda3\envs\jads_enexis\lib\site-packages\IPython\core\interactiveshell.py:3427: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['GEMEENTE', 'RES', 'M_INKHH'], dtype='object')]

  exec(code_obj, self.user_global_ns, self.user_ns)
